In [3]:
import os

In [1]:
def generate_data_generator_for_two_images(genX1):
	while True:
		X1i = genX1.next()
		yield X1i[0], X1i[1]

In [2]:
# Parameters
epochs = 20
batch_size = 20
impl = 2 			# gpu	

###########################################
# Data

img_width, img_height, channels = 224, 224, 3 		# Resolution of inputs
input_shape = 4096

dataset = 'SASE-FE'
partition = 'prealigned'


In [4]:
if dataset == 'OULU-CASIA':
	train_data_dir = os.path.join('..', '..', '_Dataset', dataset, 'consecutive', 'training')	
	validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, 'consecutive', 'validation')
	frames = 5
	n_output = 6
	nb_train_samples = 5941 / batch_size
	nb_validation_samples = 2025 / batch_size
	
else:
	partition = ''

	train_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'training')	
	validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'validation')

	frames = 5
	n_output = 12

	nb_train_samples = 27971 / batch_size
	nb_validation_samples = 4173 / batch_size


In [6]:
dataset = 'SASE-FE'
partition = 'prealigned'

train_data_dir = os.path.join('..','..', '_Dataset', dataset, '5frames', 'training')
validation_data_dir = os.path.join('..','..', '_Dataset', dataset, '5frames', 'validation')

frames = 5
n_output = 12

nb_train_samples = 27971 / batch_size
nb_validation_samples = 4173 / batch_size


In [7]:
train_data_dir

'../../_Dataset/SASE-FE/5frames/training'

In [8]:
data_dir = '/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB'

train_data_dir = os.path.join(data_dir, '5frames', 'training')
validation_data_dir = os.path.join(data_dir, '5frames', 'validation')

In [9]:
train_data_dir

'/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB/5frames/training'

In [12]:
# split the data into training and validation from the directory with videos SASE-FE
train_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'training')
validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'validation')

frames = 5

# number of classes


In [13]:
train_data_dir

'../../_Dataset/SASE-FE/5frames/training'

In [25]:
data_dir = '/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB'

participants = os.listdir(data_dir)



In [26]:
# data.remove('.DS_Store')

In [27]:
participants

['Pavel',
 'Alena',
 'Merilin',
 'yiiri',
 'Dona',
 'Victor',
 'Suman',
 'Hassan',
 'Airiin',
 'Polina',
 '.DS_Store',
 'Toyib',
 'Alexander',
 'Elmar',
 'Karl Gregori',
 'Roxanne',
 'Sameer',
 'Kirill',
 'Lukas',
 'Remo',
 'Anton',
 'Asif',
 'Nikita',
 'Vladimir',
 'Rezwan',
 'Iiris',
 'Vladimiz',
 'anee',
 'Pejman',
 'Reena',
 'age',
 'Umesh',
 'Laur',
 'Karl',
 'Laura',
 'sinle',
 'Francisca',
 'Mari-liis',
 'Nina',
 'Darwin',
 'Helen',
 'Richard',
 'Andreas',
 'Ivan',
 'Yeh',
 'Aleksander Makarov',
 'Laura Jogede',
 'Lembit',
 'Ahmed',
 'Mate',
 'Teddy']